In [1]:
import time
import pandas as pd
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [3]:
url = "https://www.amazon.com.br"

driver.get(url)
#time.sleep(5)

In [4]:
#campo de busca
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
search_box.send_keys("protetor solar facial")

#botao de busca
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()

driver.implicitly_wait(5)

In [5]:
#obter os links dos produtos
link = driver.find_elements(By.XPATH, "//a[contains(@class, 'a-link-normal s-no-outline')]")

prod_link_list = []
prod_audit = []

for lk in link:
    href_value = lk.get_attribute("href")
    prod_link_list.append(href_value)
    prod_audit.append(lk.text)

#print(prod_link_list)
#print(len(prod_link_list))
#print(prod_audit)

In [6]:
prod_price = []
prod_descr = []
prod_link = []
prod_marca = []
prod_detalhe = []
prod_comentario = []
prod_aval = []
prod_dataref = []

# Obter data, hora e minuto atual
data_hora_atual = datetime.now()

for url in prod_link_list:
    driver.get(url)
    time.sleep(5)

    try:
        price = driver.find_element(By.XPATH, "//span[contains(@class, 'a-price aok-align-center')]")
        check_price = price.text.strip()
    except:
        check_price = ''

    if check_price != '':

        prod_price.append(price.text.replace("\n", ",").replace("R$",""))

        prod_dataref.append(data_hora_atual)

        prod_link.append(url)

        descricao = driver.find_element(By.ID, "productTitle")
        prod_descr.append(descricao.text)

        try:
            marca = driver.find_element(By.XPATH, "//*[@id='detailBullets_feature_div']/ul/li[2]/span/span[2]")
            prod_marca.append(marca.text)
        except:
            prod_marca.append("SEM MARCA")

        try:
            detalhe = driver.find_element(By.ID, "productDescription_feature_div")
            prod_detalhe.append(detalhe.text)
        except:
            try:
                detalhe = driver.find_element(By.ID, "richProductDescription_feature_div")
                prod_detalhe.append(detalhe.text)
            except:
                detalhe = "SEM DETALHES"
                prod_detalhe.append(detalhe)

        try:
            comentario = driver.find_element(By.ID, "cm-cr-dp-review-list")
            prod_comentario.append(comentario.text)
        except:
            comentario = "SEM COMENTARIOS"
            prod_comentario.append(comentario)

        try:
            avaliacao = driver.find_element(By.XPATH, "//*[@id='cm_cr_dp_d_rating_histogram']/div[2]/div/div[2]/div/span/span")
            prod_aval.append(avaliacao.text)
        except:
            prod_aval.append("SEM AVALIACOES")

driver.quit()


In [7]:
tab_resultado = {
    "prod_descricao" : prod_descr,
    "prod_marca" : prod_marca,
    "prod_URL" : prod_link,
    "prod_price" : prod_price,
    "prod_detalhe" : prod_detalhe,
    "prod_comentario" : prod_comentario,
    "prod_avaliacao" : prod_aval,
    "prod_dataref" : prod_dataref
}

df = pd.DataFrame(tab_resultado)

In [8]:
print(len(prod_descr))
print(len(prod_marca))
print(len(prod_link))
print(len(prod_price))
print(len(prod_detalhe))
print(len(prod_comentario))
print(len(prod_aval))

49
49
49
49
49
49
49


In [9]:
df.to_csv("C:\\Users\\Amend\\Downloads\\AMAZON_TEMP_22082024_1640.csv", sep=';', index=False, encoding='utf-8-sig')